In [1]:
#------------FFN----------------

# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

file_path = '/content/age_gender.csv'

# Load the dataset
age_gender = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(age_gender.head())
print(age_gender.shape)
print(age_gender.dtypes)
print(age_gender.isnull().sum())
print(age_gender.describe())

# Data visualization
plt.figure(figsize=(10, 6))
sns.histplot(age_gender['age'], bins=10, kde=False)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Drop unnecessary column
age_gender.drop('img_name', axis=1, inplace=True)

# Check the lengths of the pixel sequences
pixel_lengths = age_gender['pixels'].apply(lambda x: len(x.split()))
print(pixel_lengths.value_counts())  # See if there are different lengths

# Expected length for 48x48 images
expected_length = 48 * 48

# Filter out rows with incorrect pixel lengths
age_gender = age_gender[pixel_lengths == expected_length]

# Verify that all remaining sequences have the correct length
pixel_lengths = age_gender['pixels'].apply(lambda x: len(x.split()))
print(pixel_lengths.value_counts())  # Should show only one length

# Convert the 'pixels' column to arrays
X = np.array(age_gender['pixels'].apply(lambda x: np.array(x.split(), dtype='float32')).tolist())

# Flatten arrays to vectors for FFN
X = X.reshape(-1, 48 * 48)

# Check the shape to ensure it's correct
print(X.shape)

# Extract target variables
y_gender = to_categorical(age_gender['gender'], 2)

# Split data into training and testing sets
X_train, X_test, y_train_gender, y_test_gender = train_test_split(X, y_gender, test_size=0.2, random_state=42)

# Normalize the pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Model architecture for FFN with increased complexity
model = Sequential([
    Dense(1024, activation='relu', input_shape=(48 * 48,), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 units and softmax activation for binary classification
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model with early stopping and learning rate reduction
history = model.fit(
    X_train, y_train_gender,
    epochs=50,  # Train for more epochs
    batch_size=32,
    validation_data=(X_test, y_test_gender),
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_gender)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7da8801660b6>", line 7, in <cell line: 7>
    import tensorflow as tf
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/__init__.py", line 48, in <module>
    from tensorflow._api.v2 import __internal__
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/__internal__/__init__.py", line 8, in <module>
    from tensorflow._api.v2.__internal__ import autograph
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/__internal__/autograph/__init__.py", line 8, in <module>
    from tensorflow.python.autograph.core.ag_ctx import control_status_ctx # line: 34
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/core/ag_ctx.py", line 21, in <module>
    from tensorflow.python.autograph.utils import ag_logging
  File "/us

TypeError: object of type 'NoneType' has no len()